In [ ]:
# default_exp masks

In [ ]:
#export
from speechsep.imports import *
from speechsep.plot import *
from speechsep.utils import *
from speechsep.core import *

#hide
fn_list = Path("../data/SmallDataset/esc50_sample/").glob(r"*/*")
audio_list = [AudioBase(*load(fn)) for fn in fn_list]

# Sound Separation Masks
> All AudioMasking techniques that I know of.

In [ ]:
#export
class MaskBase():
    _show_args={}
    def __init__(self, data):
        store_attr(self, 'data')
    @property
    def shape(self):
        return self.data.shape
    @classmethod
    def create(cls, specs, mix_spec):
        return [cls.generate(spec, mix_spec) for spec in specs]
    def __mul__(self, spec):
        raise NotImplementedError('This function needs to be implemented before use')
    def __rmul__(self, spec):
        return self*spec
    def generate(self, spec, mix_spec):
        raise NotImplementedError('This function needs to be implemented before use')
    @delegates(setup_graph)
    def show(self, ctx=None, **kwargs): return show_mask(self, ctx=ctx, **merge(self._show_args, kwargs))


In [ ]:
MaskBase._tensor_cls = TensorMask
@ToTensor
def encodes(self, o:MaskBase):  return o._tensor_cls(mask2tensor(o))
def mask2tensor(mask:MaskBase):  return TensorMask(mask.data)

# Mask Types

## Binary Mask

In [ ]:
#export
class MaskBinary(MaskBase):
    def __mul__(self, spec):
        new_spec = SpecImage(spec.data*self.data, spec.sr, spec.fn)
        return new_spec
    @classmethod
    def generate(cls, spec, joined):
        return (joined.data <= spec.data)*1

## Complex Ideal Ratio Mask

In [ ]:
#export
class MaskcIRM(MaskBase):
    def __mul__(self, spec):
        new_spec = SpecImage(spec.data*self.data, spec.sr, spec.fn)
        return new_spec
    @classmethod
    def generate(cls, spec, joined):
        return (joined.data <= spec.data)*1

# Maskify Transform

In [ ]:
#export
class Maskify(TupleTransform):
    as_item_force=True
    def __init__(self, MaskType=MaskBinary, Aud2Spec=Spectify):
        store_attr(self, "MaskType, Aud2Spec")
    def encodes(self, audioList)->None:
        maskList = self.MaskType.create(audioList)
        return audioList, maskList
    def decodes(self, audioList, maskList)->None:
        return [a*m for a in zip(audioList, maskList)]